In [ ]:
# This makes a fake data frame for testing the needed LMIRcam orientation retrieval
# in the Dewarp code

# created 2019 May 23 by E.S.

In [22]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
from astropy.io import fits
import asciitable
from pathlib import Path
import glob
import scipy
import os
from scipy.ndimage import gaussian_filter
from astropy.convolution import convolve, Gaussian1DKernel, interpolate_replace_nans
from lmircam_tools import process_readout
from scipy.ndimage import rotate, map_coordinates

In [23]:
# this function from Jordan Stone's LEECH code
def rot(im, angle, axis, order=3, pivot=False):
    '''rotate an image clockwise by angle [degrees] about axis.
    if pivot is true the image will pivot about the axis. otherwise
    the axis will be centered in the output image''' 
    angle*=np.pi/180.#convert to radians
    y,x = np.indices(im.shape)

    # calculate how the axis moves when pivoting from bottom left corner
    theta_axis = np.arctan2(axis[1],axis[0])
    r_axis = np.abs(axis[0]+1j*axis[1])
    yoffset = r_axis*np.sin(theta_axis) - r_axis*np.sin(theta_axis-angle)
    xoffset = r_axis*np.cos(theta_axis) - r_axis*np.cos(theta_axis-angle)

    # put the axis in the middle? 
    ycenter_offset = (1-pivot) * ((im.shape[0]/2.)-axis[1])#pivot is a bool (i.e. 0 or 1)
    xcenter_offset = (1-pivot) * ((im.shape[1]/2.)-axis[0])
    yoffset += ycenter_offset
    xoffset += xcenter_offset

    # make rotation matrix elements
    ct = np.cos(angle)
    st = np.sin(angle)

    # do the rotation 
    new_x = (ct*(x-xoffset) - st*(y-yoffset)) 
    new_y = (st*(x-xoffset) + ct*(y-yoffset))
    
    return map_coordinates(im, [new_y, new_x], order=order)

In [30]:
def make_fake():
    '''
    makes a frame with two fake stars at the same RA, rotates image to a fake PA
    '''
    
    # noise
    noise_frame = 10*np.random.rand(2048,2048)
    
    # inject stars
    sigma_x= 10
    sigma_y= 10
    x0 = 1250
    y0 = 1400
    ampl = 2000
    x, y = np.meshgrid(np.linspace(0,2048,num=2048), np.linspace(0,2048,num=2048))
    #z = np.power(y,2)
    z0 = ampl*np.exp(-((x-x0)**2/(2*sigma_x**2) + (y-y0)**2/(2*sigma_y**2)))
    z1 = ampl*np.exp(-((x-x0)**2/(2*sigma_x**2) + (y-750)**2/(2*sigma_y**2)))

    fake_frame = np.add(np.add(noise_frame,z0),z1)
    
    # rotate 19.5 deg CCW
    # derotate
    fake_rot = rot(im = fake_frame,
                              angle = -20,
                              axis = [1024,1024],
                              order = 3, pivot=False) # axis coord here is just a dummy
    
    hdr = fits.Header()
    hdr["LBT_PARA"] = -19.5
    
    fits.writeto("test_fake.fits", 
                 data = fake_rot,
                 header = hdr,
                 overwrite=True)
    
    return fake_frame, z0, z1

In [31]:
test1, test2, test3 = make_fake()

In [4]:
np.shape(test)

(2048, 2048)

In [11]:
astropy.modeling.rotations.Rotation2D(angle=45.0)